In [1]:
import requests
import urllib
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

In [2]:
import streamlit

In [3]:
import numpy as np
import datetime as dt
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [19]:
import requests
import datetime as dt
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import streamlit as st


class MeasurementTool:
    def __init__(
        self,
    ):
        self.root = "https://environment.data.gov.uk/flood-monitoring"
        self.page_url = self.root + "/id/stations/"
        self.page = requests.get(self.page_url)
        self.entries = self.page.json()["items"]
        self.stations = self.create_stations_list()
        self.data = {}

    def retrieve_measure_metadata(self, measure_url):
        measure_page = requests.get(measure_url)
        return measure_page.json()["items"]

    def retrieve_station_data(self, station_id):
        """
        Returns the last 24 hours of recordings for a given station.
        """
        station_url = self.page_url + station_id
        latest_possible_reading_time = self.get_rounded_timestamp()
        last_24_hrs = self.subtract_24_hours(latest_possible_reading_time)
        station_page = requests.get(station_url + "/readings?since=" + last_24_hrs)
        station_data = station_page.json()["items"]

        for datapoint in station_data:
            measure = "-".join(
                datapoint["measure"].split("/")[-1].split("-")[1:]
            )  # Extract measure name
            if measure not in self.data.keys():
                # initialise empty inner dict to hold timestamps,metadata and values
                self.data[measure] = {}
                self.data[measure]["timestamps"], self.data[measure]["values"] = [], []
                self.data[measure]["metadata"] = self.retrieve_measure_metadata(
                    datapoint["measure"]
                )
            timestamp = datapoint["dateTime"]
            value = datapoint["value"]
            self.data[measure]["timestamps"].append(timestamp)
            self.data[measure]["values"].append(value)

    def get_rounded_timestamp(self):
        """
        Returns the current timestamp rounded down
        to the nearest 15-minute increment."""
        now = dt.datetime.now(dt.timezone.utc)
        rounded_minutes = now.minute - (
            now.minute % 15
        )  # Round down to nearest 15-minute mark
        rounded_time = now.replace(minute=rounded_minutes, second=0, microsecond=0)
        return rounded_time.strftime("%Y-%m-%dT%H:%M:%SZ")

    def subtract_24_hours(self, timestamp: str) -> str:
        """Subtracts 24 hours from a given timestamp and returns it in the same format."""
        datetime = dt.datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
        new_dt = datetime - dt.timedelta(hours=24)
        return new_dt.strftime("%Y-%m-%dT%H:%M:%SZ")

    def create_stations_list(self):
        stations = []
        station_ids = []
        for entry in self.entries:
            stations.append(entry["@id"])
            station_ids.append(entry["stationReference"])
        return station_ids

    def plot_measure(self, df):
        """Plots the time series data."""

        df.sort_values(by="timestamps", inplace=True)
        df["timestamps"] = pd.to_datetime(df["timestamps"])

        # Plot
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.plot(df["timestamps"], df["values"], marker="o", linestyle="-")

        # Formatting
        ax.set_xlabel("Timestamp")
        ax.set_ylabel("Value")
        ax.set_title("Time Series Data")
        ax.tick_params(axis="x", rotation=45)  # Rotate x-axis labels for readability
        ax.grid()
        plt.tight_layout()
        st.pyplot(fig)  # Display the plot

    def create_name_from_metadata(self, data):
        metadata = data["metadata"]
        return metadata["parameterName"] + " - " + metadata["qualifier"]


# Create an instance of MeasurementTool
tool = MeasurementTool()

In [20]:
tool.retrieve_station_data("1491TH")

In [22]:
tool.data["metadata"]

KeyError: 'metadata'

In [320]:
times = [0, 1, 2, 3, 4, 5]
values = [5, 4, 3, 2, 1, 0]
metadata = {"id": 1, "measure": "17"}

In [322]:
dictionary = defaultdict(lambda: {"timestamps": [], "values": []})

In [324]:
dictionary["iasd"]["metadata"] = metadata

In [33]:
for i in tool.data.keys():
    print()

level-stage-i-15_min-mASD
{'@id': 'http://environment.data.gov.uk/flood-monitoring/id/measures/1491TH-level-stage-i-15_min-mASD', 'datumType': 'http://environment.data.gov.uk/flood-monitoring/def/core/datumASD', 'label': 'River Cherwell at Oxford (Kings Mill) - level-stage-i-15_min-mASD', 'latestReading': {'@id': 'http://environment.data.gov.uk/flood-monitoring/data/readings/1491TH-level-stage-i-15_min-mASD/2025-03-08T15-45-00Z', 'date': '2025-03-08', 'dateTime': '2025-03-08T15:45:00Z', 'measure': 'http://environment.data.gov.uk/flood-monitoring/id/measures/1491TH-level-stage-i-15_min-mASD', 'value': 1.532}, 'notation': '1491TH-level-stage-i-15_min-mASD', 'parameter': 'level', 'parameterName': 'Water Level', 'period': 900, 'qualifier': 'Stage', 'station': 'http://environment.data.gov.uk/flood-monitoring/id/stations/1491TH', 'stationReference': '1491TH', 'type': ['http://environment.data.gov.uk/flood-monitoring/def/core/Measure', 'http://environment.data.gov.uk/flood-monitoring/def/core

In [41]:
# measure_metadata = [["metadata"] for i in tool.data]
labels = [tool.create_name_from_metadata(tool.data[i]) for i in tool.data]

In [44]:
[i for i in tool.data]

['level-stage-i-15_min-mASD', 'level-downstage-i-15_min-mASD']

In [37]:
measure_metadata[1]

{'@id': 'http://environment.data.gov.uk/flood-monitoring/id/measures/1491TH-level-downstage-i-15_min-mASD',
 'datumType': 'http://environment.data.gov.uk/flood-monitoring/def/core/datumASD',
 'label': 'River Cherwell at Oxford (Kings Mill) - level-downstage-i-15_min-mASD',
 'latestReading': {'@id': 'http://environment.data.gov.uk/flood-monitoring/data/readings/1491TH-level-downstage-i-15_min-mASD/2025-03-08T15-45-00Z',
  'date': '2025-03-08',
  'dateTime': '2025-03-08T15:45:00Z',
  'measure': 'http://environment.data.gov.uk/flood-monitoring/id/measures/1491TH-level-downstage-i-15_min-mASD',
  'value': 0.874},
 'notation': '1491TH-level-downstage-i-15_min-mASD',
 'parameter': 'level',
 'parameterName': 'Water Level',
 'period': 900,
 'qualifier': 'Downstream Stage',
 'station': 'http://environment.data.gov.uk/flood-monitoring/id/stations/1491TH',
 'stationReference': '1491TH',
 'type': ['http://environment.data.gov.uk/flood-monitoring/def/core/Measure',
  'http://environment.data.gov.uk

In [308]:
tool = MeasurementTool()
tool.create_widgets()

Dropdown(description='Station:', options=(' ', '1029TH', 'E2043', '52119', 'E21136', '2067', '48143', '720215'…

Dropdown(description='Measure:', options=(), style=DescriptionStyle(description_width='initial'), value=None)

Button(description='Retrieve Data', style=ButtonStyle())

Output()

Fetching data for 1029TH...


In [348]:
tool.data

defaultdict(<function __main__.MeasurementTool.__init__.<locals>.<lambda>()>,
            {'metadata': {'timestamps': [], 'values': []}})

In [198]:
data = tool.retrieve_measurement("1029TH")